In [1]:
import os
import sys
import torch
import matplotlib.pyplot as plt
import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import v2

PROJECT_ROOT= "/teamspace/studios/this_studio/Relational-Group-Activity-Recognition/utils/data_utils"
sys.path.append(os.path.abspath(PROJECT_ROOT))
from data_loader import Person_Activity_DataSet, Group_Activity_DataSet

dataset_root = "/teamspace/studios/this_studio/Relational-Group-Activity-Recognition/data"
annot_path =   f"{dataset_root}/annot_all.pkl"
videos_path =  f"{dataset_root}/videos"

people_activity_clases = ["Waiting", "Setting", "Digging", "Falling" ,"Spiking"	, "Blocking", "Jumping"	, "Moving", "Standing"]
person_activity_labels  = {class_name.lower():i for i, class_name in enumerate(people_activity_clases)}

group_activity_clases = ["r_set", "r_spike" , "r-pass", "r_winpoint", "l_winpoint", "l-pass", "l-spike", "l_set"]
group_activity_labels  = {class_name:i for i, class_name in enumerate(group_activity_clases)}


train_spilt = [1, 3, 6, 7, 10, 13, 15, 16, 18, 22, 23, 31, 32, 36, 38, 39, 40, 41, 42, 48, 50, 52, 53, 54]

## Test group activity data loader

#### Can return a all player crops of the target frame with its group label (all player have same label)  

**Note:** 
1. Player_tensor.shape: [(12, 3, 244, 244)] - Label_tensor.shape [(1, 8)]
2. All 12 Player sorted by x-axis positions (need to Construct graph later)

In [2]:
transforms = A.Compose([
    A.Resize(224, 224),  
    ToTensorV2()
])

data_loader= Group_Activity_DataSet(
    videos_path,
    annot_path, 
    split=train_spilt, 
    sort=True, 
    seq=False, 
    labels=group_activity_labels, 
    transform=transforms
)

In [ ]:
len(data_loader) 

In [4]:
frame_crops, label = data_loader[152]

In [ ]:
print(label.shape) # (,8)
label

In [ ]:
frame_crops.shape # (12, C, H, W) ---> 12 bbox of the frame

In [ ]:
# Visualize each player crop 

titles = [f"Player {i+1}" for i in range(12)]  

plt.figure(figsize=(15, 5))

for i, player in enumerate(frame_crops[:6]): # take frist team
    plt.subplot(1, 6, i + 1)
    plt.imshow(player.permute(1, 2, 0).numpy())
    plt.title(titles[i])  
    plt.axis('off')      

plt.tight_layout()
plt.show()

In [ ]:
titles = [f"Player {i+1}" for i in range(12)]  

plt.figure(figsize=(15, 5))

for i, player in enumerate(frame_crops[6:]): # take second team
    plt.subplot(1, 6, i + 1)
    plt.imshow(player.permute(1, 2, 0).numpy())
    plt.title(titles[i])  
    plt.axis('off')      

plt.tight_layout()
plt.show()

##### Visualize 50 Random Sample to test player sorting

In [ ]:
import numpy as np

random_idexes = np.random.choice(range(len(data_loader)), (50), replace=False)

for sample_idex in random_idexes.tolist():
    frame_crops, label = data_loader[sample_idex]

    print('_' * 100)
    print("Visualize sample : ", sample_idex)
    # Visualize each player crop 
    print("Team One: \n\n")
    titles = [f"Player {i+1}" for i in range(12)]  

    plt.figure(figsize=(15, 5))

    for i, player in enumerate(frame_crops[:6]): # take frist team
        plt.subplot(1, 6, i + 1)
        plt.imshow(player.permute(1, 2, 0).numpy())
        plt.title(titles[i])  
        plt.axis('off')      

    plt.tight_layout()
    plt.show()

    print("Team Two: \n\n")
    titles = [f"Player {i+1}" for i in range(12)]  

    plt.figure(figsize=(15, 5))

    for i, player in enumerate(frame_crops[6:]): # take second team
        plt.subplot(1, 6, i + 1)
        plt.imshow(player.permute(1, 2, 0).numpy())
        plt.title(titles[i])  
        plt.axis('off')      

    plt.tight_layout()
    plt.show()


#### Can return a full clip with sorted player crop with its group label 

**Note:** 
1. Player_tensor.shape: [(12, 9, 3, 244, 244)] - Label_tensor.shape [(9, 8)]
2. All 12 Playersorted by x-axis positions on each single frame (needed to constract graph later)

In [13]:
transforms = A.Compose([
    A.Resize(224, 224),  
    ToTensorV2()
])

data_loader = Group_Activity_DataSet(
    videos_path,
    annot_path, 
    split=train_spilt, 
    seq=True, 
    sort=True, 
    labels=group_activity_labels, 
    transform=transforms
)

In [ ]:
clip, label = data_loader[250]
clip.shape

In [ ]:
label_idex = label[0].argmax().item() # group label at frist frame
group_label = group_activity_clases[label_idex]
print(f"Group activity label: {group_label}")

In [ ]:
# Visualize each player crop  - at frist frame

frist_frame = clip[:, 0, : , : , :] # take the frist frame

titles = [f"Player {i+1}" for i in range(12)]  

plt.figure(figsize=(15, 5))

for i, player in enumerate(frist_frame[:6]): # take frist team
    plt.subplot(1, 6, i + 1)
    plt.imshow(player.permute(1, 2, 0).numpy())
    plt.title(titles[i])  
    plt.axis('off')      

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))

for i, player in enumerate(frist_frame[6:]): # take second team
    plt.subplot(1, 6, i + 1)
    plt.imshow(player.permute(1, 2, 0).numpy())
    plt.title(titles[i + 6])  
    plt.axis('off')      

plt.tight_layout()
plt.show()

In [ ]:
clip, label = data_loader[600]
label_idex = label[0].argmax().item() # group label at frist frame
group_label = group_activity_clases[label_idex]

print(f"Group activity label: {group_label}")

In [ ]:
frist_frame = clip[:, 0, : , : , :] # take the frist frame

titles = [f"Player {i+1}" for i in range(12)]  

plt.figure(figsize=(15, 5))

for i, player in enumerate(frist_frame[:6]): # take frist team
    plt.subplot(1, 6, i + 1)
    plt.imshow(player.permute(1, 2, 0).numpy())
    plt.title(titles[i])  
    plt.axis('off')      

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))

for i, player in enumerate(frist_frame[6:]): # take second team
    plt.subplot(1, 6, i + 1)
    plt.imshow(player.permute(1, 2, 0).numpy())
    plt.title(titles[i + 6])  
    plt.axis('off')      

plt.tight_layout()
plt.show()